In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px

import datetime as dt

# import mysql.connector as cnt
import sqlite3

import requests
from bs4 import BeautifulSoup

# import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
# Firefox leistų sudėtingesnius nustatymus, bet neveikia per SNAP

from selenium import webdriver
from bs4 import BeautifulSoup
import time

print('Paskaita nr. 44, 2024-10-21')
print('Atverčia ir nuskaito AutoPlius skelbimus pagal anksčiau db išsaugotas nuorodas.')
print('Parametrų kategorijų identifikatoriai "integer primary key" tipo.')

Paskaita nr. 44, 2024-10-21
Atverčia ir nuskaito AutoPlius skelbimus pagal anksčiau db išsaugotas nuorodas.


In [59]:
# https://www.sqlitetutorial.net/sqlite-primary-key/
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# https://www.sqlite.org/lang_createtable.html

db_failas='auto_skelbimai.db'
db_connect = sqlite3.connect(db_failas)
cs = db_connect.cursor()

print(f"sqlite3.connect('{db_failas}')")

sqlite3.connect('auto_skelbimai.db')


In [26]:
cs.execute("select * from auto_skelbimai limit 2")
for (nr,href,created,updated) in cs.fetchall():
    print(nr,href)

    opcijos = Options()
    opcijos.add_argument('--incognito')
    driver = webdriver.Chrome(options=opcijos)
    driver.get(href)
    time.sleep(1) # 3 sekundėms atsidaro naršyklės langas
    source = driver.page_source
    driver.close()

    bs = BeautifulSoup(source,'html.parser').find('body')

    # <div class="larger">Skelbimas neegzistuoja</div>
    larger=bs.find('div',{'class':'larger'})
    if larger and larger.text == 'Skelbimas neegzistuoja':
        print(f'Skelbimas {nr} neegzistuoja.')
        continue

18267894 https://m.autoplius.lt/skelbimai/mercedes-benz-e250-2-5-l-sedanas-1990-dyzelinas-18267894.html
Skelbimas 18267894 neegzistuoja.
23150167 https://m.autoplius.lt/skelbimai/toyota-sienna-3-3-l-vienaturis-2004-benzinas-dujos-23150167.html


In [93]:
def parametro_išsaugojimas(nr_,p_txt_id_,p_value_,db_,cs_):
    print(f"Skelbimas:{nr_}, parametras:{p_txt_id_}, reikšmė:{p_value_}")
    sql=f"select param_cat_id, param_cat_txt from parametro_kategorija where param_cat_txt='{p_txt_id_}'"
    cs_.execute(sql)
    db_.commit()
    ats = cs_.fetchall()
    print('Rasta:',ats,len(ats),end='. ') # 
    egzistuoja=len(ats)
    if egzistuoja:
        p_id_=ats[0][0]
    else:
        sql_template='''INSERT into parametro_kategorija 
        (param_cat_txt) values (?)'''
        cs_.execute(sql_template,([p_txt_id_]))
        db_.commit()
        sql=f"select param_cat_id, param_cat_txt from parametro_kategorija where param_cat_txt='{p_txt_id_}'"
        cs_.execute(sql)
        db_.commit()
        ats = cs_.fetchall()
        print('Įterpta:',ats,end='. ') # ,len(ats),ats[0][0]
        p_id_=ats[0][0]

    print('Nustatytas id:',p_id_)
    print()

    # Sukuriama kategorija jei tokios nebuvo. 
    # Nustatomas rastos arba naujai sukurtos kategorijos id.
    # Reikia įraįrašyti skelbimo-kategorijos ryšį.

    # sql_template='''INSERT OR REPLACE into auto_skelbimai (nr,href) values (?,?)'''
    # cs.execute(sql_template,(nr,href))

# print(bs)
konteineris=bs.find('div',{'id':'container'})
# print(konteineris)

title=konteineris.find('h1',{'class':'title'}).get_text().strip()
parametro_išsaugojimas(nr,'title',title,db_connect,cs)

subtitle=konteineris.find('h3',{'class':'subtitle'}).get_text().strip()
parametro_išsaugojimas(nr,'subtitle',subtitle,db_connect,cs)

picture=konteineris.find('picture').find('source')['srcset'].strip()
parametro_išsaugojimas(nr,'picture',picture,db_connect,cs)

rida=konteineris.find('span',{'title':'Rida'}).get_text().replace('Rida:','').replace(' km','').replace(' ','').strip()
parametro_išsaugojimas(nr,'rida',rida,db_connect,cs)

parametro_išsaugojimas(nr,'tmp99','tmp',db_connect,cs)

Skelbimas:23150167, parametras:title, reikšmė:Toyota Sienna
Rasta: [(1, 'title')] 1. Nustatytas id: 1

Skelbimas:23150167, parametras:subtitle, reikšmė:3.3 l., vienatūris
Rasta: [(2, 'subtitle')] 1. Nustatytas id: 2

Skelbimas:23150167, parametras:picture, reikšmė:https://autoplius-img.dgn.lt/ann_25_330980647/toyota-sienna-3-3-l-vienaturis-2004-benzinas-dujos-0.jpg
Rasta: [(3, 'picture')] 1. Nustatytas id: 3

Skelbimas:23150167, parametras:rida, reikšmė:333000
Rasta: [(4, 'rida')] 1. Nustatytas id: 4

Skelbimas:23150167, parametras:tmp99, reikšmė:tmp
Rasta: [] 0. Įterpta: [(11, 'tmp99')]. Nustatytas id: 11



In [ ]:
db_connect.close()

In [ ]:
######################################################################
################ Žemiai - vienkartiniai dalykai ######################
######################################################################

In [12]:
######################################################################
print('Perrašau lentelę į naują db')  ################################
######################################################################

sena_db = sqlite3.connect('Testas.db')
sena_cs = sena_db.cursor()

sql_create_table = '''
create table if not exists auto_skelbimai
(
    nr integer primary key,
    href text,
    create_timestamp integer,
    update_timestamp integer
) strict
'''
sena_cs.execute(sql_create_table)

dabar=int(dt.datetime.today().timestamp()) # now(), today()
eilutės=[]
sena_cs.execute('select * from auto_skelbimai') # limit 9
for rezultatas in sena_cs.fetchall():
    eilutė=list(rezultatas)
    eilutė.append(dabar) 
    eilutė.append(dabar) 
    # print(eilutė)
    eilutės.append(eilutė)
sena_db.close()
print(f"Perskaityta {len(eilutės)} įrašų.")

nauja_db = sqlite3.connect('auto_skelbimai.db')
nauja_cs = nauja_db.cursor()

sql_template='''INSERT OR REPLACE into auto_skelbimai (nr,href,create_timestamp,update_timestamp) values (?,?,?,?)'''
nauja_cs.executemany(sql_template,eilutės)
nauja_db.commit()
nauja_db.close()

Perrašau lentelę į naują db
Perskaityta 1000 įrašų.


In [9]:
######################################################################
print('Tikriname db')  ###############################################
######################################################################

db_x = sqlite3.connect('auto_skelbimai.db')
cs_x = db_x.cursor()

cs_x.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cs_x.fetchall())

cs_x.close()
db_x.close()

Tikriname db
[('auto_skelbimai',), ('parametro_kategorija',), ('skelbimo_parametras',)]


In [31]:
######################################################################
print('Sukuriamos lentelės sqlite3 duomenųbazėje') ###################
######################################################################

sql_create_table = '''
create table if not exists auto_skelbimai
(
    nr integer primary key,
    href text,
    create_timestamp integer,
    update_timestamp integer
) strict
'''
cs.execute(sql_create_table)

sql_create_table = '''
create table if not exists parametro_kategorija
(
    param_cat_id integer primary key autoincrement,
    param_cat_txt text unique on conflict fail,
    param_description text,
    create_timestamp integer,
    update_timestamp integer
) strict
'''
cs.execute(sql_create_table)

sql_create_table = '''
create table if not exists skelbimo_parametras
(
    skelbimo_nr integer not null,
    param_cat_id integer not null,
    create_timestamp integer,
    update_timestamp integer,
    primary key (skelbimo_nr,param_cat_id)
) strict
'''
cs.execute(sql_create_table)

# https://www.sqlite.org/lang_createtable.html

Sukuriamos lentelės sqlite3 duomenųbazėje
